## Objective

download all genomes in NCBI refseq or genbank at or below a specified taxon_id

## Materials, Methods, and Functions

In [1]:
# import Mycelia

┌ Info: Precompiling Mycelia [453d265d-8292-4a7b-a57c-dce3f9ae6acd]
└ @ Base loading.jl:1342


In [3]:
# Mycelia.get_sequence(db="nuccore", accession="OP164776.1")

FASTX.FASTA.Reader{TranscodingStreams.NoopStream{IOBuffer}}(TranscodingStreams.NoopStream{IOBuffer}(<mode=idle>), 1, 1, nothing, FASTX.FASTA.Record:
  description: ""
     sequence: "", true)

In [4]:
# collect(ans)

1-element Vector{FASTX.FASTA.Record}:
 FASTX.FASTA.Record:
  description: "OP164776.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/BGD/icddrb-TND-07-1525/2022 ORF1ab polyprotein (ORF1ab), ORF1a polyprotein (ORF1ab), surface glycoprotein (S), ORF3a protein (ORF3a), envelope protein (E), membrane glycoprotein (M), ORF6 protein (ORF6), and ORF7a protein (ORF7a) genes, complete cds; ORF7b (ORF7b) and ORF8 protein (ORF8) genes, partial cds; and nucleocapsid phosphoprotein (N) and ORF10 protein (ORF10) genes, complete cds"
     sequence: "AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCT…"

In [ ]:
# TODO
# just have me call NCBI datasets CLI with taxon id
# so much simpler

In [ ]:
# default parameters
@info "define_parameters"
taxon_id = 10239
data_dir = "../../data"
# database = refseq or genbank
ncbi_database = "refseq"

In [ ]:
@info "initializing genomes directory"
genome_directory = mkpath("$data_dir/genomes")

In [ ]:
@info "initializing joint fastas directory"
joint_fastas_directory = mkpath("$data_dir/joint-fastas")

In [ ]:
# import Pkg
# Pkg.build("Mycelia")

In [ ]:
@info "import libraries"
import Pkg
pkgs = [
"DataFrames",
"ProgressMeter",
]
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

import Mycelia

In [ ]:
@info "download ncbi metadata"
ncbi_metadata = Mycelia.load_ncbi_metadata(ncbi_database)

In [ ]:
@info "get child taxon_ids"
child_taxon_ids = Set([parse(Int, line) for line in eachline(open(`taxonkit list --ids $(taxon_id) --indent ""`)) if !isempty(line)])
@assert taxon_id in child_taxon_ids

In [ ]:
@info "filter ncbi metadata down to child taxon ids"
is_right_taxa = map(taxid -> taxid in child_taxon_ids, ncbi_metadata[!, "taxid"])
is_right_taxa .|= map(taxid -> taxid in child_taxon_ids, ncbi_metadata[!, "species_taxid"])
filtered_ncbi_metadata = ncbi_metadata[is_right_taxa, :]

In [ ]:
@info "define functions"

# # normally, mkpath shouldn't error if it already exists, but something funny is going on with the fact that this is mounted with rclone
# function my_mkpath(dir)
#     if !isdir(dir)
#         mkpath(dir)
#     end
#     return dir
# end

function filetype_to_ncbi_extension(filetype)
    d = Dict(
        "fasta" => "genomic.fna.gz",
        "gff" => "genomic.gff.gz"
    )
    try
        return d[filetype]
    catch
        key_options = join(keys(d), '\n')
        error("please choose one of the following\n$key_options")
    end
end

function bgzip_recompress(infile)
    @assert occursin(r"\.gz$", infile) "file does not end with .gz, skipping..."
    @assert isfile(infile) "file $infile not found, skipping..."
    bgzip_index = infile * ".gzi"
    if isfile(bgzip_index)
        # @info "$bgzip_index already exists, skipping..."
        # continue
    else
        outfile = infile * ".bgz"
        p = pipeline(pipeline(`gzip -dc $infile`, `bgzip`), infile * ".bgz")
        run(p)
        mv(outfile, infile, force=true)
        run(`bgzip -r $infile`)
    end
    return infile
end

function download_filetype_from_ncbi_ftp_path(outdir, ftp_path, filetype; recompress_with_bgzip=false)
    filename = basename(ftp_path) * "_" * filetype_to_ncbi_extension(filetype)
    full_ftp_path = joinpath(ftp_path, filename)
    full_local_path = joinpath(outdir, filename)
    if !isfile(full_local_path)
        download(full_ftp_path, full_local_path)
    end
    if recompress_with_bgzip
        bgzip_recompress(full_local_path)
    end
    return full_local_path
end

In [ ]:
ftp_list = filtered_ncbi_metadata[!, "ftp_path"]
# ftp_list = filtered_ncbi_metadata[1:10, "ftp_path"]

In [ ]:
@info "download genomes"
ProgressMeter.@showprogress for ftp_path in ftp_list
    download_filetype_from_ncbi_ftp_path(genome_directory, ftp_path, "fasta", recompress_with_bgzip=true)
end

In [ ]:
@info "merge genomes into pangenome fasta"
joint_fasta = "$joint_fastas_directory/$taxon_id.fna.gz"
write_io = open(joint_fasta, "w")
ProgressMeter.@showprogress for ftp_path in ftp_list
    outfile = download_filetype_from_ncbi_ftp_path(genome_directory, ftp_path, "fasta")
    write(write_io, read(outfile))
end
run(`bgzip -r $joint_fasta`)
close(write_io)

## Experimental/Simulated Observations

N/A

## Analysis, Statistics, and Visualizations

N/A

## Summary of Results

N/A

## Conclusions and Future Directions

N/A